# 슈카월드 컨텐츠를 기반으로 채팅하기







## Semantic Searching
  - 데이터를 로드합니다.
  - 질문의 Embedding 을 계산합니다.
  - 계산된 Embedding 과 유사한 결과들을 탐색합니다.
  - (참조) https://platform.openai.com/docs/guides/embeddings/what-are-embeddings

In [1]:
!pip install tiktoken
!pip install openai

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


## 준비!

라이브러리 들을 읽고, API KEY 를 설정합니다.

In [2]:
import numpy as np
import pandas as pd
import openai
import tiktoken
import os

In [3]:
os.environ['OPENAI_API_KEY'] = "" # OpenAI api key를 입력하세요(문자열)

client = openai.OpenAI()

### Preview

OpenAI의 API를 사용하여 GPT 를 사용합니다.

In [4]:
prompt = "우크라이나와 러시아의 전쟁은 어떻게 되어가고 있나요?"

# GPT-4로 답변 생성
completion = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
  ]
)

# API 응답에서 답변 추출
print(completion.choices[0].message.content)

죄송합니다만, 저는 실시간 정보나 최신 뉴스를 제공할 수 없습니다. 우크라이나와 러시아 간의 현재 상황에 대한 최신 정보를 얻으려면, 신뢰할 수 있는 뉴스 출처를 확인하시는 것이 가장 좋습니다. 또한, 국제적으로 인정받은 뉴스 기구가 정확한 시점에 상황을 가장 정확하게 반영할 수 있습니다.


## 슈카월드 데이터 읽기

준비된 데이터를 읽습니다.
영상에 대한 내용 (글) 과 이를 임베딩하여 벡터로 치환한 값들이 저장되어 있습니다.

In [5]:
datafile_path = "./data/shuka_embeddings.csv"

df = pd.read_csv(datafile_path)
df["embedding"] = df.embedding.apply(eval).apply(np.array)
df

,Unnamed: 0,id,title,publishTime,description,transcript,summarized,combined,embedding
0,0,ivfBP9XempU,제주도 보다 큰 미국 농장의 위엄,2023-09-29T09:18:11Z,0:00 ~ 11:18 1조원에 팔린 미국 농장의 위엄 11:18 ~ 전세계 땅부자...,[음악] 미국의 1조원 농장이 팔렸다 이거 보셨습니까 꽤 오래된 얘기긴 해요 아직도...,미국의 1조 농장이 팔렸는데 한국인들은 상상하기 힘든 크기로 비행기로 찍으면 되지 ...,Title: 제주도 보다 큰 미국 농장의 위엄; Content: 미국의 1조 농장이...,"[0.006102622990667008, -0.014742583866790107, ..."
1,1,JhcYEf319YA,가슴 아픈 이마트 시가총액 2조원,2023-09-28T10:44:36Z,0:00 ~ 14:01 이게 말이 되나 싶은 이마트 시총 14:01 ~ 23:10 ...,[음악] 자 첫 번째 주제는 이마트 시가총액이 이종원이다라는 주제로 시작을 해보도록...,이마트 시가총액이 이 씨라는 주제로 시작해 볼 텐데 이 씨는 1위 유통업체 중에서 ...,Title: 가슴 아픈 이마트 시가총액 2조원; Content: 이마트 시가총액이 ...,"[-0.0011725858647948365, -0.003846474179674443..."
2,2,kJBsrOS3rzo,"특검, 특검, 특검, 미 대선을 흔드는 사법 리스크",2023-09-27T12:05:16Z,0:00 ~ 16:25 혼돈의 미국 정치 16:25 ~ 1999년 퓰리처 상 사진 ...,"자, 봅시다 자, 그 다음은 더 얘기하기 힘든 주제입니다 아니, 이거는 솔직히 오해...",미국 메카시 하원의장이 바이든 대통령을 탄핵하기 위해 바이든 탄핵 조사를 실시했다 ...,"Title: 특검, 특검, 특검, 미 대선을 흔드는 사법 리스크; Content: ...","[-0.00939853975668415, -0.009666877079379067, ..."
3,3,6Jz5YIqUpMc,"역대 최대 금액, 3천억 원 횡령 발생",2023-09-26T11:13:36Z,0:00 ~ 10:03 우리나라 횡령계 GOAT 10:03 ~ 13:17 라임 사건...,"자, 봅시다 그 다음은 이제 또 또 안 좋은 내용이네 역대 1위라고 불리우는 3천억...",역대 1위라고 불리는 3천억 횡령 사건이 대한민국에서 또 발생했다 은행에서 발생한 ...,"Title: 역대 최대 금액, 3천억 원 횡령 발생; Content: 역대 1위라고...","[-0.011554287368376432, -0.011808374510157043,..."
4,4,D3QUDebEats,"2030년 세계 GDP 3위, 인도의 야망",2023-09-25T12:24:55Z,0:00 ~ 5:18 G20 정상회의를 쥐고 흔든 인도 5:18 ~ 13:59 인도...,"자, 마지막 주제는 오늘의 메인 주제라고 할 수 있습니다 제목은 용의 시대가 지나고...","G20 정상회의는 미국, 러시아, 한국, 중국, 일본, 영국, 프랑스, 독일, 브라...","Title: 2030년 세계 GDP 3위, 인도의 야망; Content: G20 정...","[0.008519676672105264, -0.02860960231180057, 0..."
...,...,...,...,...,...,...,...,...,...
488,495,EV-kYc7w-qE,"장난으로 만든 도지코인, 초유의 시총 50조원 돌파",2021-04-19T11:11:00Z,"어렵고 딱딱한 경제,시사,금융 이야기를 쉽고 유쾌하게 풀어내는 경제/시사/이슈/잡썰...",으 out 도지 코인 이라는게 급등하면서 시청 50조 를 돌파했다 전 이걸 처음 보...,도지 코인이 급등하면서 시청 50조 를 돌파했다 시청 50조 를 돌파한 것은 미국의...,"Title: 장난으로 만든 도지코인, 초유의 시총 50조원 돌파; Content: ...","[-0.0047661971330641125, -0.00631537639465818,..."
489,496,wF-kBWmlb2I,"힘든 청춘들, 서로 사랑하기를 응원합니다.",2021-04-12T10:38:42Z,"어렵고 딱딱한 경제,시사,금융 이야기를 쉽고 유쾌하게 풀어내는 경제/시사/이슈/잡썰...",으 out 두 번째 주제는 채팅창을 잠깐 다들 까 생각을 하고 있는데 예 최대 야 ...,우리 젊은 20대 30대 생각을 들어 보면 뉴스를 보거나 신문을 보더라도 세대 갈등...,"Title: 힘든 청춘들, 서로 사랑하기를 응원합니다.; Content: 우리 젊은...","[-0.005508284529859694, -0.013253675914015453,..."
490,497,gnnJkTl7bCo,푸틴 종신집권의 꿈과 압도적인 러시아 청년 지지율 86%,2021-04-08T10:31:47Z,"어렵고 딱딱한 경제,시사,금융 이야기를 쉽고 유쾌하게 풀어내는 경제/시사/이슈/잡썰...",으 out 러시아의 푸틴 대통령이 2021 년 3 월 26 일 입니다 그 선거법 개...,러시아의 푸틴 대통령이 선거법 개정안이 통과됐고 헌법 개정을 통해 선거법 개정안이 ...,Title: 푸틴 종신집권의 꿈과 압도적인 러시아 청년 지지율 86%; Conten...,"[0.010583974648036551, -0.006855907997017619, ..."
491,498,vc9kDKQAzys,"&#39;볼츠바겐&#39;, 역사에 남을 만우절 장난의 뜨거운 후폭풍",2021-04-06T10:56:08Z,"어렵고 딱딱한 경제,시사,금융 이야기를 쉽고 유쾌하게 풀어내는 경제/시사/이슈/잡썰...",으 out 이게 지난 후에 정말 커다란 전 세계에서 가장 큰 만우절 장난에 하나 있...,역사에 남을 만우절 장난이 발생했는데 우리의 이름은 볼 치바 긴 이라고 공식 홈페이...,"Title: &#39;볼츠바겐&#39;, 역사에 남을 만우절 장난의 뜨거운 후폭풍;...","[-0.02736300506219729, -0.027753711685491515, ..."


## 텍스트 임베딩

OPENAI 의 api 를 이용하여 프롬프트를 임베딩 해봅시다.

In [6]:
emb = client.embeddings.create(
    input=prompt,
    model='text-embedding-ada-002')
emb

CreateEmbeddingResponse(data=[Embedding(embedding=[0.010791467502713203, -0.01973259262740612, 0.03304184973239899, -0.023605097085237503, -0.021794402971863747, -0.0015612273709848523, -0.020644547417759895, 0.013798275962471962, -0.018635602667927742, -0.015542886219918728, -0.005845102947205305, -0.0006405988242477179, -0.009621786884963512, -0.02367118000984192, 0.006287863943725824, -0.00157031393609941, 0.010428007692098618, -0.026380611583590508, 0.02507215552031994, -0.02593124285340309, 0.010104197077453136, -0.004252485930919647, -0.0047712428495287895, -0.011016152799129486, -0.0194814745336771, 0.02528362348675728, 0.006535677704960108, -0.012185834348201752, 0.01167038083076477, -0.011115278117358685, 0.011392829939723015, 0.006466289982199669, 0.005359387956559658, -0.010613041929900646, -0.003518956946209073, -0.004371436778455973, 0.015199250541627407, -0.015397502109408379, 0.0068000126630067825, -0.009423535317182541, -0.006671149283647537, 0.010817901231348515, 0.016

## 임베딩으로 이용한 검색

임베딩을 만들어서 준비된 데이터와 임베딩을 비교합니다.  
cosine 을 계산하여 벡터간 거리를 측정합니다.  
가장 가까운 순으로 정렬하여 검색된 데이터를 관찰해봅니다.  

OpenAI 에서는 이를 Sementic Search 라고 부릅니다.

나만의 검색엔진, 추천 시스템이 구축되었습니다.

In [7]:
from scipy.spatial.distance import cosine

# 연관된 영상을 검색합니다.
def search_video(df, question, n=3, pprint=True):
    # 질문에 대한 임베딩 생성
    q_embeddings = client.embeddings.create(input=question, model='text-embedding-ada-002').data[0].embedding
    
    df["distance"] = df["embedding"].apply(lambda x: cosine(q_embeddings, x))
    
    results = (
        df.sort_values("distance", ascending=True)
        .head(n)
    )

    return results

In [8]:
# 하고 싶은 말을 입력하고, 비슷한 결과들을 열람합니다.
results = search_video(df, prompt, n=5)
results

,Unnamed: 0,id,title,publishTime,description,transcript,summarized,combined,embedding,distance
255,257,0q2U1Dgmpgg,세계가 놀란 우크라이나의 반격,2022-09-13T10:15:38Z,0:00 ~ 5:05 예상치 못한 우크라이나의 반격 5:05 ~ 14:20 미국의 ...,우크라이나의 반격이 러시아와 다른 모든 국가를 놀라게 했다 지금 서구 대표적인 언론...,러시아가 쇼킹상태라고 하는 우크라이나의 반격 때문에 어떤 일이 있었는지 살펴보도록 ...,Title: 세계가 놀란 우크라이나의 반격; Content: 러시아가 쇼킹상태라고 ...,"[-0.015291125870511263, -0.0131246627577784, 0...",0.104921
133,133,bO96Pvd8jx0,한국식 휴전 모델? 교착상태에 빠지는 우크라이나 전쟁,2023-03-08T11:57:19Z,0:00 ~ 7:58 바이든 대통령 깜짝 방문 7:58 ~ 14:09 부산스러운 미...,자 얼마 전에 무슨 날이었냐? 바로 어느새 우크라이나 전쟁 1주기였습니다 2022년...,우크라이나 전쟁 1주기 때 슈카만 1주기 방송할 생각으로 마음이 설렜냐는 질문에 전...,Title: 한국식 휴전 모델? 교착상태에 빠지는 우크라이나 전쟁; Content:...,"[-0.008166731693582762, -0.0225195596563055, 0...",0.113772
243,245,VL696LKF5d4,아 제발.. 고조되는 러시아발 핵위협,2022-10-05T10:26:57Z,"어렵고 딱딱한 경제,시사,금융 이야기를 쉽고 유쾌하게 풀어내는 경제/시사/이슈/잡썰...",푸틴이 우크라이나의 영토합병을 서명했습니다 서명하면서 연설도 했죠 뭐라고했냐 이것은...,푸틴이 우크라이나의 영토합병에 서명하면서 연설을 했는데 러시아 국민 수백만 명의 의...,Title: 아 제발.. 고조되는 러시아발 핵위협; Content: 푸틴이 우크라이...,"[-0.018262905206603192, -0.015384221712463737,...",0.131864
283,288,dtlA94a3TSI,"형제여.. 폴란드, 대한민국 무기 약 20조원 구매",2022-08-01T11:22:16Z,0:00 ~ 11:29 세계 2차 대전에 책임이 있는 폴란드? 11:29 ~ 19:...,폴란드가 형제의 나라로 급부상하고 있다 [박수] 야야~ 언제 형제가 되었어 야~ 몰...,러시아가 우크라이나를 침공하면서 시작된 일이 2022년 2월 24일 러시아가 우크라...,"Title: 형제여.. 폴란드, 대한민국 무기 약 20조원 구매; Content: ...","[-0.020319655037094226, -0.026926821040133858,...",0.136956
101,101,xX8ZwBTAwxM,나으리 우리는 수출만 했을 뿐입니다.,2023-04-24T11:12:14Z,0:00 ~ 12:14 러시아가 한국의 무기 제공(?)에 대해 위협했다. 12:14...,러시아가 한국을 위협한 얘기를 뉴스식으로 빨리 진행해 보도록 하겠습니다 다 보셨죠?...,러시아가 한국의 우크라이나 무기 제공에 대해 위협을 했는데 한국이 우크라이나에 무기...,Title: 나으리 우리는 수출만 했을 뿐입니다.; Content: 러시아가 한국의...,"[-0.015947872144958, -0.009386615458136165, 0....",0.139640


## 검색 결과와 함께 질문하기!
  - 질문과 유사한 컨텐츠를 검색합니다.
  - GPT 의 프롬프트에 유사한 결과를 함께 넣어 줍니다.
  - GPT 가 학습할 때 없었던 정보이지만, 이렇게 같이 넣어주면 GPT 가 누락되지 않은 정보를 이용하여 대답을 할 수 있습니다.

In [9]:
GPT_MODEL="gpt-4"
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""
    
    results = search_video(df, query, n=3)
    strings = results['combined']
    
    introduction = 'Use the below articles on the 슈카월드 to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."'
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string in strings:
        next_article = f'\n\n:\n"""\n{string}\n"""'
        print("Tokens in Use : ", num_tokens(message + next_article + question, model=model))
        if (
            num_tokens(message + next_article + question, model=model)
            > token_budget
        ):
            
            break
        else:
            message += next_article
    return message + question

def ask(
    query: str,
    df: pd.DataFrame = df,
    model: str = GPT_MODEL,
    token_budget: int = 8192 - 500,
    print_message: bool = False,
) -> str:
    """Answers a query using GPT and a dataframe of relevant texts and embeddings."""
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)
        
    completion = client.chat.completions.create(
      model=model,
      messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": message}
      ]
    )

    # API 응답에서 답변 추출
    return completion.choices[0].message.content

In [10]:
# set print_message=True to see the source text GPT was working off of
answer = ask('우크라이나와 러시아의 전쟁은 어떻게 되어가고 있나요?', model="gpt-4", print_message=True)

Tokens in Use :  1357
Tokens in Use :  2662
Tokens in Use :  3613
Use the below articles on the 슈카월드 to answer the subsequent question. If the answer cannot be found in the articles, write "I could not find an answer."

:
"""
Title: 세계가 놀란 우크라이나의 반격; Content: 러시아가 쇼킹상태라고 하는 우크라이나의 반격 때문에 어떤 일이 있었는지 살펴보도록 하자 하르키우라는 곳에서 벌어진 우크라이나의 군대에 갑작스러운 반격의 성공이 전략에 대한 분석을 바꾸고 있다는 기사가 있다 우크라이나 수도 동부는 하루 만에 돌파하고 몰도바까지 후루룩 해서 우크라이나를 내륙국가로 만들겠다는 원대한 계획을 얘기했지만 현실은 아직도 이러고 있다. 우크라이나군은 이지움이라는 전략도시를 쳐들어가서 러시아의 보급선을 끊는 것을 목표로 하고 있다 러시아의 보급선을 끊는 것을 아래로 보급선을 끊는 것을 목표로 하고 있다 우크라이나 군대가 지움에 Enter 입장하고 있는 상황이기 때문에 우크라이나 군대가 실시간으로 지움에 Enter 입장하고 있는 상황이다. 미국 국무장관 블링컨이 우크라이나 키이우를 깜짝 방문했는데 유럽에 대한 미국의 군사원조를 20억 달러로 늘리고 절반 이상을 우크라이나에 지원하겠다고 얘기했다. 미국은 우크라이나가 필요로 하는 것을 확실하게 지원하기로 결심했고 미국의 최고위층은 우크라이나의 진격을 승인했고 나토 본부에 와서 블링컨이 연설을 했다 미국이 우크라이나 전쟁에 지원한 지원 총액이 152억 달러를 넘었다 푸틴은 2023년까지 러시아군이 13만 7천명을 증원하겠다고 발표했고 우리나라가 아덴만 해적에 청해 보내 특별 군사작전을 하듯이 우크라이나에서도 특별 군사작전을 하고 있다. 러시아는 아무것도 잃은 것이 없고 러시아는 땅에 미사일 하나 넘어온 것이 없으며 우리가 가진 것을 잃지도 

In [11]:
answer

'우크라이나와 러시아 사이의 전쟁 상황은 복잡하게 전개되고 있습니다. 러시아는 우크라이나의 주요 도시인 바흐무트를 공격하여 7개월간 포위하였으며, 이 도시의 인구는 대부분 사라져 전파될 수 있는 경제적 피해를 입혔습니다. 이에 대한 반격으로, 우크라이나 군대는 러시아의 보급선을 끊는 것을 목표로 하여 전략적으로 반격하고 있습니다. \n\n또한, 미국은 이 상황에 대해 우크라이나를 지원하는 입장을 유지하고 있습니다. 미국 국무장관 블링컨은 20억 달러의 군사원조를 늘려 절반 이상을 우크라이나에 지원하겠다고 선언하였고, 우크라이나에 제공한 총 지원금액은 152억 달러를 넘어섰습니다. \n\n그러나 불구하고, 러시아는 핵무기 사용 가능성을 언급하며 국제사회로부터 우려의 목소리를 자아냈습니다. 이에 따라, 전쟁의 상황이 계속 교착상태로 빠지면서 양 측의 휴전 가능성도 논의되고 있습니다. 한편, 중국 또한 이 사안에 개입하려는 움직임을 보여, 우크라이나의 전쟁 상황은 더욱 복잡해질 가능성이 있습니다.'